# Netbox Demo - Devops Team - 08/07/2023

## Make the Connection

In [ ]:
import pynetbox
import json
import os
import pandas as pd
import requests

In [ ]:
nb_url = "https://demo.netbox.dev"
print(f"First go here and create a user account: {nb_url}/plugins/demo/login/")

In [ ]:
nb = pynetbox.api(nb_url)

In [ ]:
token = nb.create_token("pdemo", "abcd1234")
print(nb.token)

## View System Version

In [ ]:
nb.status()

In [ ]:
nb.version

In [ ]:
list(nb.plugins.installed_plugins())

#Find plugins here: https://github.com/netbox-community/netbox/wiki/Plugins

## Retrieve and Inspect Devices

In [ ]:
devices = nb.dcim.devices.all()

In [ ]:
devices

In [ ]:
devicelist = list(devices)

In [ ]:
mydev = devicelist[0]

In [ ]:
mydev.serialize()

In [ ]:
(mydev.id, mydev.name, mydev.device_type.manufacturer, mydev.device_type.model, mydev.device_role.name)

In [ ]:
mydev.device_type.manufacturer.serialize()

In [ ]:
cisco_routers = [dev for dev in devicelist if dev.device_type.manufacturer.slug == "cisco" and dev.device_role.slug == "router"]

In [ ]:
cisco_routers

In [ ]:
mycisco = cisco_routers[0]

In [ ]:
(mycisco.id, mycisco.name, mycisco.device_type.manufacturer, mycisco.device_type.model, mycisco.device_role.name)

## Edit a Device (Example: Add Comment)

In [ ]:
# Set a comment on a device
device_name = "dmi01-akron-rtr01"
new_comment = "Comment from pdemo"
try:
    device = nb.dcim.devices.get(name=device_name)
    if not device:
        print(f"Device with name '{device_name}' not found.")
    else:
        # Update the comment
        device.comments = new_comment
        device.save()
except pynetbox.RequestError as e:
    print(f"Failed to set comment on device: {e}")

# Check that comment took
device = nb.dcim.devices.get(name=device_name)
try:
    assert device.comments == new_comment
except AssertionError:
    print(f"Comment NOT set on device 'id: {device.id} name: {device_name}': comment: {new_comment}")
else:
    print(f"Comment set on device 'id: {device.id} name: {device_name}': comment: {new_comment}")

print(f"Check results at https://demo.netbox.dev/dcim/devices/{device.id}")

## Edit a Config Template and Render the Config for Cisco Routers

In [ ]:
# Define a template
cfg_template_name = "my_config_template"
mytemplate = ('hostname {{ device.name }}\n'
              '{% for server in ntp_servers %}\n'
              'ntp server {{ server }}\n'
              '{% endfor %}\n')

# Create the config template
try:
    # Verify the name isn't already taken (make this idempotent)
    existing_names = [ct.name for ct in nb.extras.config_templates.all()]
    assert cfg_template_name not in existing_names
    # Create the new config template
    nct = nb.extras.config_templates.create(name=cfg_template_name, template_code=mytemplate)
    print(f"Config template '{nct.name}' with ID {nct.id} created successfully!")
except AssertionError:
    print(f"Config Template Name {cfg_template_name} already exists.")
except pynetbox.RequestError as e:
    print(f"Failed to create config template: {e}")

In [ ]:
config_context = """
{
  "ntp_servers": [
                   "172.16.10.22",
                   "172.16.10.33"
   ]
}
"""

jcc = json.loads(config_context)

In [ ]:
ctid = nb.extras.config_templates.get(name=cfg_template_name).id

In [ ]:
for device in cisco_routers:
    device.local_context_data = jcc
    device.config_template = ctid
    try:
        # Save the context locally to the router
        #print(device.serialize())
        success = device.save()
        assert success
        print(f"Configuration updated on device id {device.id} name {device.name}")
    except (AssertionError, pynetbox.RequestError) as e:
        print(f"Failed to update configuration on device id {device.id} name {device.name} because {e.__class__}: {e}")

In [ ]:
print(f"Check results at https://demo.netbox.dev/dcim/devices/{device.id}/render-config/")

## Get Device Types and Roles

In [ ]:
device_types = nb.dcim.device_types.all()
device_types_dict = {dt.id: dt for dt in device_types}
devices = nb.dcim.devices.all()
devices_list = list(devices)

In [ ]:
dt_output_list = []
for id, dev_type in device_types_dict.items():
    device_count = len([dev for dev in devices_list if int(dev.device_type.id) == int(id)])
    dt_output_list.append([id, dev_type.display, device_count])

In [ ]:
df = pd.DataFrame(dt_output_list, columns=["ID", "Display Name", "Device Count"])
df.set_index("ID", inplace=True)

In [ ]:
df.sort_values("Device Count", ascending=False)

In [ ]:
device_roles = nb.dcim.device_roles.all()
device_roles_dict = {dr.id: dr for dr in device_roles}

In [ ]:
dt_output_list = []
for id, dev_role in device_roles_dict.items():
    device_count = len([dev for dev in devices_list if int(dev.device_role.id) == int(id)])
    dt_output_list.append([id, dev_role.display, device_count])

In [ ]:
df = pd.DataFrame(dt_output_list, columns=["ID", "Display Name", "Device Count"])
df.set_index("ID", inplace=True)

In [ ]:
df.sort_values("Device Count", ascending=False)

## Use Requests Library (Raw REST) Instead of Pynetbox

In [ ]:
headers = {'Authorization': 'Token {}'.format(token.key), 
           'Content-Type': 'application/json'}

In [ ]:
jdata = requests.get("https://demo.netbox.dev/api/dcim/devices/1", headers=headers)

In [ ]:
jdata.json()

## Use Ansible to View and Update

In [ ]:
os.environ["NETBOX_URL"] = nb_url
os.environ["NETBOX_TOKEN"] = nb.token

In [ ]:
%pycat lookup.yml

In [ ]:
!ansible-playbook lookup.yml